In [1]:
import os
import math
import re
import functools
import itertools
from abc import ABC
from dataclasses import dataclass, field
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt

Add any general utility functions as static member functions to this class. They can be called as `Util.fn`.

In [2]:
class Util:
    def __init__(self):
        raise TypeError("Non-instantiable class")
        
    def __new__(self, *args, **kwargs):
        raise TypeError("Non-instantiable class")
        
    @staticmethod
    def make_regex_group_disjunction(coll):
        return "|".join(map(lambda x: f"({str(x)})", coll))

In [18]:
class OptionsData:
    data_dir = "data"
    default_filename = "isx2010C.xls"
    
    def __init__(self, filename=default_filename, clean=True):
        filepath = os.path.join(self.data_dir, filename)
        if not os.path.isfile(filepath):
            faulty_filepath = filepath
            filepath = os.path.join(self.data_dir, self.default_filename)
            print(f"Warning: could not find {faulty_filepath!r}; proceeding with {filepath!r}")
        self.__sheet_df_dict = pd.read_excel(filepath, sheet_name=None)
        if clean:
            for key, val in self.__sheet_df_dict.items():
                self.__sheet_df_dict[key] = self.__clean_df(val)
                
    def __get_item__(self, key):
        return self.__sheet_df_dict[key]
    
    def get_sheet_df_dict(self):
        return self.__sheet_df_dict
    
    def get_df(self, E=None, sheet_name=""):
        if not sheet_name:
            sheet_name = list(self.__sheet_df_dict.keys())[0]
            print(f"Warning: sheet name not specified; proceeding with {sheet_name!r}")
        df = self.__sheet_df_dict[sheet_name]
        common = ["T", "T_norm", "S", "r"]
        if not E:
            return df[[*common, *filter(lambda x: re.match(r"[0-9]+", x), df.columns)]]
        strikes = E if type(E) is list or type(E) is tuple else [E]
        cols = [*common, *map(lambda x: str(int(x)), strikes)]
        return df[cols]
    
    def __clean_df(self, df):
        # Discard rows where no options data is available.
        df = df.dropna(how="all")
        # Rename the columns according to the following convention:
        #  T = Time to Maturity
        #  S = Price of the Underlying
        #  r = Risk-Free Interest Rate
        df = df.rename(lambda x: self.__rename_df_cols(str(x), df), axis="columns")
        # Adjust the interest rate properly.
        df["r"] = df["r"] / 100
        # Add new column with annual-normalized T (252 = no. trading days in a year).
        df["T_norm"] = df["T"] / 252
        # Re-arrange the columns.
        common = ["S", "r", "T", "T_norm"]
        cols = [*common, *filter(lambda x: re.search("[0-9]+", x), df.columns.astype(str))]
        return df[cols]
    
    def __rename_df_cols(self, col_name, df):
        ncol = len(df.columns)
        # Time to maturity | (price of the underlying | risk-free rate).
        regex = r"(?P<T>[0-9]+(-[0-9]{2}){2} ([0-9]{2}:){2}[0-9]{2})|(?P<Sr>Unnamed: (?P<idx>[0-9]+))"
        match = re.match(regex, col_name)
        if not match:
            return col_name
        if match["T"]:
            return "T"
        elif match["Sr"]:
            col_idx = int(match["idx"])
            # Third last depicts the price of the underlying...
            if col_idx == ncol - 3:
                return "S"
            # ...and the second last the risk free rate.
            elif col_idx == ncol - 2:
                return "r"


In [19]:
data = OptionsData("isx2010C.xls")
data.get_df().tail()

,T,T_norm,S,r,340,345,350,355,360,365,370,375,380,385,390,395,400,405,410,415,420,425,430,435,440,445,450,455,460,465,470,475,480,485,490,495,500,505,510,515,520,525,530,535,540,545,550,555,560,565,570
81,5,0.019841,524.29,0.0006,185.50,180.50,175.50,170.50,165.50,160.50,155.50,150.50,145.50,140.50,135.50,130.50,125.50,120.50,115.50,110.50,105.50,100.50,95.50,90.50,85.40,80.40,75.40,70.40,65.4,60.40,55.40,50.45,45.45,40.45,35.45,30.50,25.55,20.00,15.70,11.0,6.8,3.00,1.00,0.30,0.10,0.07,0.05,0.05,0.05,0.05,0.05
82,4,0.015873,527.93,0.0006,188.55,183.55,178.55,173.55,168.55,163.55,158.55,153.55,148.55,143.55,138.55,133.55,128.55,123.55,118.55,113.55,108.55,103.55,98.55,93.55,88.55,83.55,78.55,73.55,68.7,63.55,58.55,53.55,48.60,43.65,38.65,33.65,28.60,23.65,18.65,14.0,9.0,4.50,1.60,0.40,0.15,0.07,0.05,0.05,0.05,0.15,0.15
83,3,0.011905,529.59,0.0006,190.10,185.10,180.10,175.10,170.10,165.10,160.10,155.10,150.10,145.10,140.10,135.10,130.10,125.10,120.10,115.10,110.10,105.10,100.10,95.10,90.10,85.10,80.10,75.10,70.1,65.10,60.10,55.10,50.30,45.10,40.10,35.10,30.80,26.00,20.00,15.7,10.1,5.65,1.95,0.40,0.05,0.05,0.02,0.05,0.05,0.15,0.05
84,2,0.007937,524.11,0.0005,184.10,179.10,174.10,169.10,164.10,159.10,154.10,149.10,144.10,139.10,134.10,129.10,124.10,119.10,114.10,109.10,104.10,99.10,94.10,89.10,84.10,79.10,74.10,69.10,63.1,59.10,54.10,49.10,45.50,41.25,34.58,27.80,24.00,19.40,14.10,9.1,4.1,0.05,0.05,0.04,0.05,0.05,0.01,0.05,0.05,0.05,0.05
85,1,0.003968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A class encapsulating the Black-Scholes-Merton model and related computations, such as Greeks. Can create instances from `pd.Series` objects (as returned by pd.DataFrame.iterrows) via the `BSM.make_from_series` factory method.

In [24]:
@dataclass(frozen=True)
class BSM:
    S: float
    E: float
    r: float
    T: float
    C_obs: float
    sigma: float = 1.0
    d1: float = field(init=False)
    d2: float = field(init=False)
    
    def __post_init__(self):
        S, E, r, T, C_obs, sigma = self.S, self.E, self.r, self.T, self.C_obs, self.sigma
        sigma, *_ = minimize(self.__implied_vol_objective, sigma, args=(S, E, r, T, C_obs))['x']
        object.__setattr__(self, "sigma", sigma)
        eps = np.finfo(float).eps
        d1 = (math.log(S / E) + (r + 0.5 * self.sigma**2) * T) / (sigma * math.sqrt(T) + eps)
        object.__setattr__(self, "d1", d1)
        d2 = self.d1 - math.sqrt(self.sigma**2 * T)
        object.__setattr__(self, "d2", d2)
        
    @staticmethod    
    def make_from_dict(d, E):
        return BSM(d["S"], int(E), d["r"], d["T_norm"], d[E])
    
    @staticmethod
    def make_from_series(ser, E, sigma=1.0):
        ser = ser.filter(regex=Util.make_regex_group_disjunction(["S", int(E), "r", "T_norm"]), axis="index")
        assert ser.shape[0] == 4, f"Error: The series should have an index of form [S, E, r, T_norm], got {ser.index}."
        S, r, T, C_obs = ser.array
        return BSM(S, E, r, T, C_obs, sigma=sigma)
    
    @functools.cached_property
    def delta(self):
        return norm.cdf(self.d1)
    
    @functools.cached_property
    def gamma(self):
        return norm.pdf(self.d1) / (self.S * self.sigma * math.sqrt(self.T))
    
    @functools.cached_property
    def theta(self):
        S, E, r, T, sigma, d1, d2 = self.S, self.E, self.r, self.T, self.sigma, self.d1, self.d2
        return -0.5 * S * norm.pdf(d1) * sigma / math.sqrt(T) - r * E * math.exp(-r * T) * norm.cdf(d2)
    
    @functools.cached_property
    def vega(self):
        return self.S * math.sqrt(self.T) * norm.pdf(self.d1)
    
    def __implied_vol_objective(self, sigma0, S, E, r, T, C_obs):
        eps = np.finfo(float).eps
        d1 = (math.log(S / E) + (r + 0.5 * sigma0**2) * T) / (math.sqrt(sigma0**2 * T) + eps)
        d2 = d1 - math.sqrt(sigma0**2 * T)
        C = norm.cdf(d1) * S - E * math.exp(-r * T) * norm.cdf(d2)
        return 0.5 * (C - C_obs)**2

In [27]:
test = data.get_df()
td = test.to_dict("index")
for t, row in td.items():
    print(f"Day {t}")
    row = {k: v for k, v in row.items() if not math.isnan(v)}
    strikes = list(filter(lambda x: re.match(r"[0-9]+", x), row.keys()))
    BSMs = {}
    for E in strikes:
        BSMs[E] = BSM.make_from_dict(row, E)
    greeks = pd.DataFrame(index=strikes)
    greeks["delta"] = pd.Series([BSMs[E].delta for E in greeks.index], index=greeks.index)
    greeks["gamma"] = pd.Series([BSMs[E].gamma for E in greeks.index], index=greeks.index)
    greeks["vega"] = pd.Series([BSMs[E].vega for E in greeks.index], index=greeks.index)
    print(greeks[greeks > 0.001].fillna(0.0))
    print("-" * 50)

Day 0
        delta  gamma        vega
340  0.000000    0.0    0.000000
360  0.000000    0.0    0.000000
380  0.000000    0.0    0.000000
400  0.000000    0.0    0.000000
420  0.000000    0.0    0.000000
440  0.000000    0.0    0.000000
460  0.000000    0.0    0.000000
480  0.399126    0.0  110.829038
500  0.536640    0.0  114.026100
520  1.000000    0.0    0.000000
--------------------------------------------------
Day 1
        delta  gamma        vega
340  0.000000    0.0    0.000000
360  0.000000    0.0    0.000000
380  0.000000    0.0    0.000000
400  0.000000    0.0    0.000000
420  0.000000    0.0    0.000000
440  0.000000    0.0    0.000000
460  0.000000    0.0    0.000000
480  0.379191    0.0  109.246323
500  0.516185    0.0  114.444834
520  1.000000    0.0    0.000000
--------------------------------------------------
Day 2
        delta  gamma        vega
340  0.000000    0.0    0.000000
360  0.000000    0.0    0.000000
380  0.000000    0.0    0.000000
400  0.000000    0.0  

        delta  gamma        vega
340  0.000000    0.0    0.000000
350  0.000000    0.0    0.000000
360  0.000000    0.0    0.000000
370  0.000000    0.0    0.000000
380  0.000000    0.0    0.000000
390  0.000000    0.0    0.000000
400  0.000000    0.0    0.000000
410  0.000000    0.0    0.000000
420  0.000000    0.0    0.000000
430  0.000000    0.0    0.000000
440  0.000000    0.0    0.000000
450  0.000000    0.0    0.000000
460  0.000000    0.0    0.000000
470  0.000000    0.0    0.000000
480  0.000000    0.0    0.000000
490  0.000000    0.0    0.000000
500  0.421838    0.0  101.468613
510  0.511391    0.0  103.418547
520  0.603463    0.0   99.961593
530  1.000000    0.0    0.000000
540  1.000000    0.0    0.000000
550  1.000000    0.0    0.000000
560  1.000000    0.0    0.000000
570  1.000000    0.0    0.000000
--------------------------------------------------
Day 21
        delta  gamma        vega
340  0.000000    0.0    0.000000
350  0.000000    0.0    0.000000
360  0.000000    0

        delta  gamma       vega
340  0.000000    0.0   0.000000
350  0.000000    0.0   0.000000
360  0.000000    0.0   0.000000
370  0.000000    0.0   0.000000
380  0.000000    0.0   0.000000
390  0.000000    0.0   0.000000
400  0.000000    0.0   0.000000
410  0.000000    0.0   0.000000
420  0.000000    0.0   0.000000
430  0.000000    0.0   0.000000
440  0.000000    0.0   0.000000
450  0.000000    0.0   0.000000
460  0.000000    0.0   0.000000
470  0.000000    0.0   0.000000
480  0.433235    0.0  88.460958
490  0.512934    0.0  89.673204
500  1.000000    0.0   0.000000
510  1.000000    0.0   0.000000
520  1.000000    0.0   0.000000
530  1.000000    0.0   0.000000
540  1.000000    0.0   0.000000
550  1.000000    0.0   0.000000
560  1.000000    0.0   0.000000
570  1.000000    0.0   0.000000
--------------------------------------------------
Day 33
        delta  gamma       vega
340  0.000000    0.0   0.000000
350  0.000000    0.0   0.000000
360  0.000000    0.0   0.000000
370  0.000000 

        delta  gamma       vega
340  0.000000    0.0   0.000000
350  0.000000    0.0   0.000000
360  0.000000    0.0   0.000000
370  0.000000    0.0   0.000000
380  0.000000    0.0   0.000000
390  0.000000    0.0   0.000000
400  0.000000    0.0   0.000000
410  0.000000    0.0   0.000000
420  0.000000    0.0   0.000000
430  0.000000    0.0   0.000000
440  0.000000    0.0   0.000000
450  0.000000    0.0   0.000000
460  0.000000    0.0   0.000000
470  0.000000    0.0   0.000000
480  0.000000    0.0   0.000000
490  0.000000    0.0   0.000000
500  0.381065    0.0  80.337360
510  0.480598    0.0  84.003650
520  0.590417    0.0  81.933738
530  1.000000    0.0   0.000000
540  1.000000    0.0   0.000000
550  1.000000    0.0   0.000000
560  1.000000    0.0   0.000000
570  1.000000    0.0   0.000000
--------------------------------------------------
Day 44
        delta  gamma       vega
340  0.000000    0.0   0.000000
350  0.000000    0.0   0.000000
360  0.000000    0.0   0.000000
370  0.000000 

        delta  gamma       vega
340  0.000000    0.0   0.000000
345  0.000000    0.0   0.000000
350  0.000000    0.0   0.000000
355  0.000000    0.0   0.000000
360  0.000000    0.0   0.000000
365  0.000000    0.0   0.000000
370  0.000000    0.0   0.000000
375  0.000000    0.0   0.000000
380  0.000000    0.0   0.000000
385  0.000000    0.0   0.000000
390  0.000000    0.0   0.000000
395  0.000000    0.0   0.000000
400  0.000000    0.0   0.000000
405  0.000000    0.0   0.000000
410  0.000000    0.0   0.000000
415  0.000000    0.0   0.000000
420  0.000000    0.0   0.000000
425  0.000000    0.0   0.000000
430  0.000000    0.0   0.000000
435  0.000000    0.0   0.000000
440  0.000000    0.0   0.000000
445  0.000000    0.0   0.000000
450  0.000000    0.0   0.000000
455  0.000000    0.0   0.000000
460  0.000000    0.0   0.000000
465  0.000000    0.0   0.000000
470  0.000000    0.0   0.000000
475  0.000000    0.0   0.000000
480  0.000000    0.0   0.000000
485  0.000000    0.0   0.000000
490  0.0

...

        delta  gamma       vega
340  0.000000    0.0   0.000000
345  0.000000    0.0   0.000000
350  0.000000    0.0   0.000000
355  0.000000    0.0   0.000000
360  0.000000    0.0   0.000000
365  0.000000    0.0   0.000000
370  0.000000    0.0   0.000000
375  0.000000    0.0   0.000000
380  0.000000    0.0   0.000000
385  0.000000    0.0   0.000000
390  0.000000    0.0   0.000000
395  0.000000    0.0   0.000000
400  0.000000    0.0   0.000000
405  0.000000    0.0   0.000000
410  0.000000    0.0   0.000000
415  0.000000    0.0   0.000000
420  0.000000    0.0   0.000000
425  0.000000    0.0   0.000000
430  0.000000    0.0   0.000000
435  0.000000    0.0   0.000000
440  0.000000    0.0   0.000000
445  0.000000    0.0   0.000000
450  0.000000    0.0   0.000000
455  0.000000    0.0   0.000000
460  0.000000    0.0   0.000000
465  0.000000    0.0   0.000000
470  0.000000    0.0   0.000000
475  0.000000    0.0   0.000000
480  0.000000    0.0   0.000000
485  0.000000    0.0   0.000000
490  0.0

        delta  gamma       vega
340  0.000000    0.0   0.000000
345  0.000000    0.0   0.000000
350  0.000000    0.0   0.000000
355  0.000000    0.0   0.000000
360  0.000000    0.0   0.000000
365  0.000000    0.0   0.000000
370  0.000000    0.0   0.000000
375  0.000000    0.0   0.000000
380  0.000000    0.0   0.000000
385  0.000000    0.0   0.000000
390  0.000000    0.0   0.000000
395  0.000000    0.0   0.000000
400  0.000000    0.0   0.000000
405  0.000000    0.0   0.000000
410  0.000000    0.0   0.000000
415  0.000000    0.0   0.000000
420  0.000000    0.0   0.000000
425  0.000000    0.0   0.000000
430  0.000000    0.0   0.000000
435  0.000000    0.0   0.000000
440  0.000000    0.0   0.000000
445  0.000000    0.0   0.000000
450  0.000000    0.0   0.000000
455  0.000000    0.0   0.000000
460  0.000000    0.0   0.000000
465  0.000000    0.0   0.000000
470  0.000000    0.0   0.000000
475  0.000000    0.0   0.000000
480  0.000000    0.0   0.000000
485  0.000000    0.0   0.000000
490  0.0

        delta  gamma       vega
340  0.982252    0.0   4.570888
345  0.981740    0.0   4.683018
350  0.981205    0.0   4.799543
355  0.980646    0.0   4.920777
360  0.980059    0.0   5.047072
365  0.979444    0.0   5.178821
370  0.978797    0.0   5.316445
375  0.000000    0.0   0.000000
380  0.000000    0.0   0.000000
385  0.000000    0.0   0.000000
390  0.000000    0.0   0.000000
395  0.000000    0.0   0.000000
400  0.000000    0.0   0.000000
405  0.000000    0.0   0.000000
410  0.000000    0.0   0.000000
415  0.000000    0.0   0.000000
420  0.000000    0.0   0.000000
425  0.000000    0.0   0.000000
430  0.000000    0.0   0.000000
435  0.000000    0.0   0.000000
440  0.000000    0.0   0.000000
445  0.000000    0.0   0.000000
450  0.000000    0.0   0.000000
455  0.000000    0.0   0.000000
460  0.000000    0.0   0.000000
465  0.000000    0.0   0.000000
470  0.000000    0.0   0.000000
475  0.000000    0.0   0.000000
480  0.000000    0.0   0.000000
485  0.000000    0.0   0.000000
490  0.0

        delta  gamma       vega
340  0.985462    0.0   2.452006
345  0.985043    0.0   2.512610
350  0.984606    0.0   2.575592
355  0.984149    0.0   2.641127
360  0.983669    0.0   2.709401
365  0.983167    0.0   2.780625
370  0.982639    0.0   2.855026
375  0.982083    0.0   2.932860
380  0.981497    0.0   3.014414
385  0.980878    0.0   3.099996
390  0.980223    0.0   3.189984
395  0.979528    0.0   3.284751
400  0.978790    0.0   3.384772
405  0.978003    0.0   3.490561
410  0.977163    0.0   3.602687
415  0.976263    0.0   3.721843
420  0.975295    0.0   3.848793
425  0.974253    0.0   3.984447
430  0.000000    0.0   0.000000
435  0.000000    0.0   0.000000
440  0.000000    0.0   0.000000
445  0.000000    0.0   0.000000
450  0.000000    0.0   0.000000
455  0.000000    0.0   0.000000
460  0.000000    0.0   0.000000
465  0.000000    0.0   0.000000
470  0.000000    0.0   0.000000
475  0.000000    0.0   0.000000
480  0.000000    0.0   0.000000
485  0.000000    0.0   0.000000
490  0.0

<ipython-input-27-63c0d43af9e8>:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  greeks["delta"] = pd.Series([BSMs[E].delta for E in greeks.index], index=greeks.index)
<ipython-input-27-63c0d43af9e8>:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  greeks["gamma"] = pd.Series([BSMs[E].gamma for E in greeks.index], index=greeks.index)
<ipython-input-27-63c0d43af9e8>:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  greeks["vega"] = pd.Series([BSMs[E].vega for E in greeks.index], index=greeks.index)


In [21]:
E = 340
schedule = 2
df = data.get_df(E).dropna()

import time

t0 = time.time()
# t = 0:
rows_iterator = df.iterrows()
_, row = next(rows_iterator)
bsm_prev = BSM.make_from_series(row, E)
long_prev = bsm_prev.C_obs
delta_factor = bsm_prev.delta
short_prev = delta_factor * bsm_prev.S

# 0 < t < T:
mse = 0.0
for t, row in rows_iterator:
    bsm = BSM.make_from_series(row, E)
    long = bsm.C_obs
    dlong = long - long_prev
    short = delta_factor * bsm.S
    dshort = short - short_prev
    mse += (dlong - dshort)**2
    long_prev = long
    bsm_prev = bsm
    # Rehedge?
    if t % schedule == 0:
        delta_factor = bsm.delta
        short_prev = delta_factor * bsm.S
    else:
        short_prev = short

mse /= df.shape[0]

t1 = time.time()
print(f"Took {(t1 - t0)*1000:.2f} ms")

Took 167.94 ms


In [22]:
print(f"Single option delta hedging {mse=:.2f}")

Single option delta hedging mse=21.88
